# Part 8 bis - 协议介绍

### 背景 

现在我们已经完成了计划，我们将引入一个称为**协议**的新对象。协议协调一系列计划，并将其部署在远方的工作人员上并一次性运行它们。

它是一个高级对象，其中包含分布在多个工作程序中的复杂计算的逻辑。协议的主要特征是能够在工作人员之间发送/搜索/取回，并最终部署到确定的工作人员中。因此，用户可以设计协议，然后将其上载到Cloud Worker，其他任何Worker都可以搜索，下载并在其所连接的Worker上应用其包含的计算程序。

让我们看看如何使用它！

作者:
- Théo Ryffel - Twitter [@theoryffel](https://twitter.com/theoryffel) - GitHub: [@LaRiffle](https://github.com/LaRiffle)

中文版译者：
- Hou Wei - github：[@dljgs1](https://github.com/dljgs1)

### 1. 创建和部署

通过提供`（worker,plan）`对的列表来创建协议。“worker”可以是工作机，也可以是工人ID，也可以是表示虚拟工人的字符串。在创建时可以使用后一种情况来指定部署时同一工作机应拥有（或不拥有）两个计划。“计划”可以是计划，也可以是PointerPlan（计划指针）。

In [ ]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False

让我们定义3个计划并将其提供给协议。 它们都执行增量操作。

In [ ]:
@sy.func2plan(args_shape=[(1,)])
def inc1(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc2(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc3(x):
    return x + 1

protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])

现在我们需要将协议绑定到工作机，这是通过调用`.deploy（* workers）`完成的。让我们创建一些工作机。

In [ ]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")

In [ ]:
workers = alice, bob, charlie

protocol.deploy(*workers)

您会看到计划已经发送给适当的工作人员：已经部署好了！

这已分两个阶段完成：首先，我们将创建时提供的虚拟工作程序（以字符串命名）映射到提供的工作程序，其次，我们将相应的计划发送给每个工作机。

### 2. 运行协议

运行协议意味着依次执行所有计划。这样做，您提供了一些输入数据，该数据将发送到第一个计划位置。运行第一个计划，并将其输出移到第二个计划位置，依此类推。 运行完所有计划后，将返回最终结果，该结果由指向最后一个计划位置的指针组成。

In [ ]:
x = th.tensor([1.0])
ptr = protocol.run(x)
ptr

In [ ]:
ptr.get()

输入1.0经历了3个计划，因此已经增加了3倍，这就是现在等于4.0的原因！

实际上，您还可以在某些指向数据的指针上**远程运行协议**：

In [ ]:
james = sy.VirtualWorker(hook, id="james")

In [ ]:
protocol.send(james)

In [ ]:
x = th.tensor([1.0]).send(james)
ptr = protocol.run(x)
ptr

如您所见，结果是一个指向James的指针

In [ ]:
ptr = ptr.get()
ptr

In [ ]:
ptr = ptr.get()
ptr

### 3. 搜索协议

在实际设置中，您可能希望下载一个远程协议，将其部署在您的工作程序上并与数据一起运行：

让我们初始化一个**尚未部署的协议**，然后将其放在远程工作器上

In [ ]:
protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])
protocol.tag('my_protocol')
protocol.send(james)

In [ ]:
me = sy.hook.local_worker # 以本地工作机的身份与我联系

现在我们开始搜索以找到协议

In [ ]:
responses = me.request_search(['my_protocol'], location=james)
responses

您有权访问协议的指针

In [ ]:
ptr_protocol = responses[0]

像通常的指针一样，您可以将其取回：

In [ ]:
protocol_back = ptr_protocol.get()
protocol_back

我们可以像在第1和第2部分中所做的那样进行操作。

In [ ]:
protocol_back.deploy(alice, bob, charlie)

x = th.tensor([1.0])
ptr = protocol_back.run(x)
ptr.get()

协议将附带更多实际示例，但您已经可以看到此新对象带来的所有可能性！

# 恭喜!!! 是时候加入社区了!

祝贺您完成本笔记本教程！ 如果您喜欢此方法，并希望加入保护隐私、去中心化AI和AI供应链（数据）所有权的运动，则可以通过以下方式做到这一点！

### 给 PySyft 加星

帮助我们的社区的最简单方法是仅通过给GitHub存储库加注星标！ 这有助于提高人们对我们正在构建的出色工具的认识。

- [Star PySyft](https://github.com/OpenMined/PySyft)

### 选择我们的教程

我们编写了非常不错的教程，以更好地了解联合学习和隐私保护学习的外观，以及我们如何为实现这一目标添砖加瓦。

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### 加入我们的 Slack!

保持最新进展的最佳方法是加入我们的社区！ 您可以通过填写以下表格来做到这一点[http://slack.openmined.org](http://slack.openmined.org)

### 加入代码项目!

对我们的社区做出贡献的最好方法是成为代码贡献者！ 您随时可以转到PySyft GitHub的Issue页面并过滤“projects”。这将向您显示所有概述，选择您可以加入的项目！如果您不想加入项目，但是想做一些编码，则还可以通过搜索标记为“good first issue”的GitHub问题来寻找更多的“一次性”微型项目。

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### 捐赠

如果您没有时间为我们的代码库做贡献，但仍想提供支持，那么您也可以成为Open Collective的支持者。所有捐款都将用于我们的网络托管和其他社区支出，例如黑客马拉松和聚会！

[OpenMined's Open Collective Page](https://opencollective.com/openmined)